In [ ]:
import pickle
import os.path
from typing import Tuple, Dict, Set, List

boxes_dict: Dict[str, Tuple[Tuple[Tuple[int, int, int, int],...],  Tuple[str,...]]]
with open(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'Generated_Files', 'Boxes_dictionary.pkl'), 'rb') as f:
    boxes_dict = pickle.load(f)

all_labels: Set[str] = set()
for k in boxes_dict.keys():
    new_labels = []
    for l in boxes_dict[k][1]:
        if l == 'Blumen in Vase' or l == 'Blüte' or l == 'Blumenstrauß' or l == 'Blumen' or l == 'Blüten':
            new_labels.append('Blumen')
        elif l == 'Schloss':
            new_labels.append('Burg')
        elif l == 'Berg':
            new_labels.append('Gebirge')
        elif l == 'Pferd' or l == 'Esel':
            new_labels.append('Pferd oder Esel')
        elif l == 'Hütte' or l == 'Haus' or l == 'Ruine':
            new_labels.append('Gebäude')
        elif l == 'Pfad':
            new_labels.append('Weg')
        elif l == 'Rucksack' or l == 'Tasche':
            new_labels.append('Tasche')
        elif l == 'Turban' or l == 'Mütze' or l == 'Hut':
            new_labels.append('Kopfbedeckung')
        elif l == 'Körbe':
            new_labels.append('Korb')
        elif l == 'Wasser' or l == 'Meer':
            new_labels.append('Gewässer')
        elif l == 'Gesicht':
            new_labels.append('Kopf')
        elif l == 'Schiff':
            new_labels.append('Boot')
        elif l == 'Apfel' or l == 'Birne' or l == 'Tomate' or l == 'Kürbis':
            new_labels.append('Obst')
        elif l == 'Schaf' or l == 'Ziege' or l == 'Ziegenbock':
            new_labels.append('Schaf oder Ziege')
        else:
            new_labels.append(l)
    for l in new_labels:
        all_labels.add(l)
    boxes_dict[k] = (boxes_dict[k][0], tuple(new_labels))

occurrence_dict: Dict[str, int] = dict()

for l in all_labels:
    occurrence_dict[l] = 0
    for k in boxes_dict.keys():
        for current in boxes_dict[k][1]:
            if current == l:
                occurrence_dict[l] += 1
                break
to_delete: List[str] = []

for k in boxes_dict.keys():
    boxes: Tuple[Tuple[int, int, int, int]] = boxes_dict[k][0]
    labels: Tuple[str] = boxes_dict[k][1]
    new_boxes: List[Tuple[int, int, int, int]] = []
    new_labels: List[str] = []
    for b, l in zip(boxes, labels):
        if occurrence_dict[l] < 50:
            continue
        new_boxes.append(b)
        new_labels.append(l)
    if len(new_boxes) == 0:
        to_delete.append(k)
    else:
        boxes_dict[k] = (tuple(new_boxes), tuple(new_labels))

for k in to_delete:
    del boxes_dict[k]

all_labels: Set[str] = set()
for k in boxes_dict.keys():
    for l in boxes_dict[k][1]:
        all_labels.add(l)

to_delete = []
for k in occurrence_dict.keys():
    if k not in all_labels:
        to_delete.append(k)
for l in to_delete:
    del occurrence_dict[l]

print(occurrence_dict)
print(len(all_labels))

with open('cleaned_bounding_boxes.pkl', 'wb+') as f:
    pickle.dump(boxes_dict, f)

In [ ]:
from Project.Utils.Misc.Nlp import NLP

nlp: NLP = NLP.instance
for label in all_labels:
    assert sum(nlp.nlp(label).vector) != 0